In [14]:
# Get apikey
import dotenv
import os

dotenv.load_dotenv()
apikey = os.getenv("AI-API-KEY")

In [24]:
from langchain_openai.llms import OpenAI
from langchain_ollama import OllamaLLM

try:
    model = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=apikey,
        model="mistralai/mistral-small-3.2-24b-instruct:free",
        #temperature = 0.7
        #max_tokens = 256
    )
    model.invoke("The sky is")
except:
    model = OllamaLLM(
        base_url="http://localhost:11434",
        model = "deepseek-r1:8b",
        temperature= 0.8,
        max_tokens = 2
    )

In [25]:
model.invoke("The sky is")

"<think>\n\n</think>\n\nThe sky is a vast, mostly empty space that covers the Earth in all directions, from the top of the stratosphere to the horizon and beyond. It includes the Earth's atmosphere, which has various layers like the troposphere, stratosphere, mesosphere, and ionosphere. The sky varies in appearance depending on factors like weather conditions, time of day, and atmospheric composition.\n\nIf you're referring to something specific about the sky, feel free to clarify!"

In [32]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

In [38]:
model = ChatOllama(
    base_url="http://localhost:11434",
    model = "deepseek-r1:8b",
    reasoning=False,
)

In [39]:
human_message = HumanMessage("What is the capital of United Kingdom")
system_message = SystemMessage(""" 
You are a helpful assitant that answer queries with one word inside exclamations marks"""
                               )

In [40]:
prompt = [system_message, human_message]

In [41]:
model.invoke(prompt)

AIMessage(content='<think>\nOkay, so I\'m trying to figure out the capital of the United Kingdom. I remember hearing it\'s London, but I want to make sure I\'m not mixing anything up. Maybe it\'s a common mistake? I should check my facts.\n\nWait, the user mentioned "London" as the capital in their query. That seems straightforward, but maybe there are nuances. I know that sometimes people confuse the capital with other cities like Edinburgh or Cardiff, which are significant locations but not capitals. For instance, Edinburgh is the capital of Scotland, and Cardiff is the capital of Wales. But when it comes to the entire UK, London is definitely the main one.\n\nLet me think about historical contexts too. London has been a central city for centuries, especially during the rise of the British Empire. It\'s often associated with significant events in world history, which further solidifies its status as the capital. I don\'t recall any changes to this status, so it should still be London